In [38]:
from mace.calculators import mace_mp
from ase import build

from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
from ase import Atoms
from ase.build import bulk
from ase.visualize import view
import numpy as np
from ase.build import add_vacuum
from ase.optimize import LBFGS, BFGS, FIRE
from ase import Atom

from ase.io.trajectory import Trajectory
from ase.md import Langevin, Bussi
from ase import io


#macemp = mace_mp(dispersion=True, default_dtype="float64")
macemp_omat = mace_mp(model="/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/mace-omat-0-medium.model", dispersion=True, default_dtype="float64")


Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections


/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


In [7]:
import importlib
import interlayer_spacing
importlib.reload(interlayer_spacing)

from interlayer_spacing import calculate_interlayer_spacing, set_interlayer_spacing

In [ ]:
mxene_relaxed = io.read("structures/mxene_relaxed_omat.xyz")
graphene_atoms = io.read("structures/graphene_atoms.xyz")
go_relax = io.read("structures/go_omat_relax.xyz")
goh_relax = io.read("structures/goh_omat_relax.xyz")
m_g_relaxed = io.read("structures/m_g_relaxed_omat.xyz")
m_go_relaxed = io.read("structures/m_go_relaxed_omat.xyz")
m_goh_relaxed = io.read("structures/m_goh_relaxed_omat.xyz")

In [25]:
def create_heterostructure(mxene, graphene_layer, height):
    # Stack MXene and graphene-based layer with a given interlayer spacing (height)
    graphene_layer = graphene_layer.copy()
    
    mxene_avg_top_O_z = np.mean([atom.position[2] for atom in mxene if atom.tag == 3])

    #mxene_avg_top_C_z = np.mean([atom.position[2] for atom in mxene if atom.symbol == 'C'])
    #mxene_avg_top_O_z = np.mean([atom.position[2] for atom in mxene if atom.symbol == 'O' and atom.position[2] > mxene_avg_top_C_z])
    
    graphene_layer.positions[:, 2] += mxene_avg_top_O_z - np.average(graphene_layer.positions[:, 2]) + height
    heterostructure = mxene + graphene_layer
    print("MXene avg top O z: ", mxene_avg_top_O_z)

    return heterostructure


m_g = create_heterostructure(mxene_relaxed, graphene_atoms, 4.5)
m_go = create_heterostructure(mxene_relaxed, go_relax, 4.5)
m_goh = create_heterostructure(mxene_relaxed, goh_relax, 4.5)


MXene avg top O z:  12.23008024
MXene avg top O z:  12.23008024
MXene avg top O z:  12.23008024


In [27]:
m_go_relaxed = m_go.copy()
m_go_relaxed.calc = macemp_omat
optimiser = BFGS(m_go_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 14:34:41    -1198.835207       10.046930
BFGS:    1 14:34:44    -1200.634240        8.461493
BFGS:    2 14:34:48    -1204.844879        2.631950
BFGS:    3 14:34:52    -1205.232598        1.899181
BFGS:    4 14:34:55    -1205.460562        0.577139
BFGS:    5 14:34:58    -1205.543908        0.391042
BFGS:    6 14:35:02    -1205.576397        0.250424
BFGS:    7 14:35:05    -1205.588557        0.175854
BFGS:    8 14:35:09    -1205.606484        0.223429
BFGS:    9 14:35:13    -1205.623452        0.199752
BFGS:   10 14:35:17    -1205.631721        0.155647
BFGS:   11 14:35:21    -1205.637441        0.134718
BFGS:   12 14:35:25    -1205.645536        0.205826
BFGS:   13 14:35:28    -1205.660140        0.281952
BFGS:   14 14:35:31    -1205.681481        0.352793
BFGS:   15 14:35:35    -1205.707373        0.347516
BFGS:   16 14:35:38    -1205.736309        0.340715
BFGS:   17 14:35:41    -1205.768530        0.364606
BFGS:   18 14:

True

In [28]:
io.write("structures/m_go_relaxed_omat.xyz", m_go_relaxed)

In [39]:
m_goh_relaxed = m_goh.copy()
m_goh_relaxed.calc = macemp_omat
optimiser = FIRE(m_goh_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
FIRE:    0 16:18:28    -1202.638638       10.021121
FIRE:    1 16:18:31    -1207.387820        3.400068
FIRE:    2 16:18:34    -1207.700069        3.885068
FIRE:    3 16:18:37    -1208.157520        2.744816
FIRE:    4 16:18:40    -1208.636670        1.334973
FIRE:    5 16:18:43    -1208.761057        1.693636
FIRE:    6 16:18:46    -1208.785100        1.593013
FIRE:    7 16:18:50    -1208.828221        1.400314
FIRE:    8 16:18:53    -1208.882042        1.133429
FIRE:    9 16:18:57    -1208.937431        1.041391
FIRE:   10 16:19:00    -1208.987525        0.970673
FIRE:   11 16:19:04    -1209.030121        0.924794
FIRE:   12 16:19:07    -1209.067883        0.905174
FIRE:   13 16:19:10    -1209.109650        0.902364
FIRE:   14 16:19:14    -1209.159785        0.904117
FIRE:   15 16:19:17    -1209.217460        0.899085
FIRE:   16 16:19:21    -1209.272287        0.886739
FIRE:   17 16:19:24    -1209.307284        0.881739
FIRE:   18 16:

True

In [52]:
io.write("structures/m_goh_relaxed_omat.xyz", m_goh_relaxed)

In [53]:
m_g_relaxed_omat = io.read("structures/m_g_relaxed_omat.xyz")
m_go_relaxed_omat = io.read("structures/m_go_relaxed_omat.xyz")
m_goh_relaxed_omat = io.read("structures/m_goh_relaxed_omat.xyz")

In [54]:
view(m_goh_relaxed_omat, viewer="x3d")

In [55]:
def add_sodium(heterostructure):
    
    heterostructure = heterostructure.copy()
    
    graphene_C_atoms = [atom.position[2] for atom in heterostructure if atom.tag == 10]
    mxene_O_atoms = [atom.position[2] for atom in heterostructure if atom.tag == 3]
    
    z_mxene = np.mean(mxene_O_atoms)
    z_graphene = np.mean(graphene_C_atoms)

    # place Na ion halfway between MXene and graphene
    na_z = z_mxene + 0.5 * (z_graphene - z_mxene)

    # find the center of the heterostructure to place Na
    positions = heterostructure.get_positions()
    x_center = np.mean(positions[:, 0])  # x mean
    y_center = np.mean(positions[:, 1])  # y mean


    na_position = [x_center, y_center, na_z]

    # Add Na atom
    heterostructure += Atom('Na', na_position)
    # add tag
    heterostructure[-1].tag = 20

    return heterostructure

m_g_na = add_sodium(m_g_relaxed_omat)
m_go_na = add_sodium(m_go_relaxed_omat)
m_goh_na = add_sodium(m_goh_relaxed_omat)


In [43]:
view(m_goh_na, viewer='x3d')

In [12]:
m_g_na_relaxed = m_g_na.copy()
m_g_na_relaxed.calc = macemp_omat
optimiser = BFGS(m_g_na_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 13:36:03    -1205.239156        1.754752
BFGS:    1 13:36:05    -1205.349633        1.150601
BFGS:    2 13:36:07    -1205.432894        0.638756
BFGS:    3 13:36:09    -1205.448924        0.576722
BFGS:    4 13:36:11    -1205.500513        0.358416
BFGS:    5 13:36:13    -1205.516533        0.339016
BFGS:    6 13:36:15    -1205.531591        0.351237
BFGS:    7 13:36:17    -1205.546852        0.354096
BFGS:    8 13:36:19    -1205.563217        0.327107
BFGS:    9 13:36:21    -1205.576942        0.270634
BFGS:   10 13:36:24    -1205.589816        0.202088
BFGS:   11 13:36:26    -1205.602272        0.192741
BFGS:   12 13:36:28    -1205.612207        0.200772
BFGS:   13 13:36:30    -1205.619096        0.139503
BFGS:   14 13:36:32    -1205.624464        0.102388
BFGS:   15 13:36:34    -1205.629022        0.105512
BFGS:   16 13:36:36    -1205.632934        0.096267
BFGS:   17 13:36:38    -1205.636191        0.079704
BFGS:   18 13:

True

In [ ]:
io.write("structures/m_g_na_relaxed_omat.xyz", m_g_na_relaxed)

In [34]:
m_go_na_relaxed = m_go_na.copy()
m_go_na_relaxed.calc = macemp_omat
optimiser = BFGS(m_go_na_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 15:50:07    -1208.695616        2.663659
BFGS:    1 15:50:10    -1208.869517        2.248450
BFGS:    2 15:50:13    -1209.140223        1.167313
BFGS:    3 15:50:16    -1209.200211        0.821774
BFGS:    4 15:50:19    -1209.274801        0.682655
BFGS:    5 15:50:23    -1209.306210        0.528754
BFGS:    6 15:50:26    -1209.325239        0.530923
BFGS:    7 15:50:30    -1209.345269        0.482388
BFGS:    8 15:50:33    -1209.366670        0.422595
BFGS:    9 15:50:35    -1209.384443        0.490266
BFGS:   10 15:50:38    -1209.400987        0.424327
BFGS:   11 15:50:41    -1209.420795        0.384354
BFGS:   12 15:50:44    -1209.440506        0.337160
BFGS:   13 15:50:47    -1209.456112        0.227291
BFGS:   14 15:50:51    -1209.468578        0.229372
BFGS:   15 15:50:54    -1209.479871        0.231428
BFGS:   16 15:50:57    -1209.490237        0.212867
BFGS:   17 15:51:00    -1209.498624        0.174641
BFGS:   18 15:

True

In [44]:
io.write("structures/m_go_na_relaxed_omat.xyz", m_go_na_relaxed)

In [56]:
m_goh_na_relaxed = m_goh_na.copy()
m_goh_na_relaxed.calc = macemp_omat
optimiser = BFGS(m_goh_na_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 22:04:23    -1213.534745        3.792228
BFGS:    1 22:04:25    -1213.830913        2.926336
BFGS:    2 22:04:27    -1214.233071        1.373870
BFGS:    3 22:04:29    -1214.325843        0.816736
BFGS:    4 22:04:31    -1214.388528        0.494055
BFGS:    5 22:04:33    -1214.419919        0.410111
BFGS:    6 22:04:37    -1214.438252        0.389521
BFGS:    7 22:04:39    -1214.453528        0.351630
BFGS:    8 22:04:42    -1214.469117        0.299835
BFGS:    9 22:04:44    -1214.482647        0.264000
BFGS:   10 22:04:46    -1214.493503        0.242132
BFGS:   11 22:04:50    -1214.505606        0.351337
BFGS:   12 22:04:52    -1214.519083        0.343259
BFGS:   13 22:04:55    -1214.531311        0.204098
BFGS:   14 22:04:57    -1214.541292        0.149430
BFGS:   15 22:05:00    -1214.549492        0.125985
BFGS:   16 22:05:02    -1214.556151        0.142353
BFGS:   17 22:05:04    -1214.561101        0.106966
BFGS:   18 22:

True

In [57]:
io.write("structures/m_goh_na_relaxed_omat.xyz", m_goh_na_relaxed)

In [61]:
m_g_na_relaxed_omat = io.read("structures/m_g_na_relaxed_omat.xyz")
m_go_na_relaxed_omat = io.read("structures/m_go_na_relaxed_omat.xyz")
m_goh_na_relaxed_omat = io.read("structures/m_goh_na_relaxed_omat.xyz")

In [62]:
view(m_goh_na_relaxed_omat, viewer='x3d')

In [16]:
from ase.constraints import FixSubsetCom

import time
start_time = time.time()

def printenergy(a=m_g_na_relaxed_omat):
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    elapsed_time = time.time() - start_time  # Time in seconds
    elapsed_min, elapsed_sec = divmod(elapsed_time, 60)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV Time Elapsed: %dm %.1fs' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin, int(elapsed_min), int(elapsed_sec)))



# m_g
#m_g_relaxed.calc = macemp
m_g_na_relaxed_omat.calc = macemp_omat
T_init = 300  # Initial temperature in K
MaxwellBoltzmannDistribution(m_g_na_relaxed_omat, temperature_K=T_init)
#remove_COM_momentum_xy(m_g_relaxed)
#Stationary(m_g_relaxed)

tags = m_g_na_relaxed_omat.get_tags()

mxene_indices = np.where(np.isin(tags, [0, 1, 2, 3]))[0]  # MXene atoms
graphene_indices = np.where(np.isin(tags, [10, 11, 12]))[0]  # Graphene atoms
m_g_na_relaxed_omat.set_constraint(FixSubsetCom(mxene_indices))
m_g_na_relaxed_omat.set_constraint(FixSubsetCom(graphene_indices))

#dyn = Langevin(m_g_relaxed, 1 * units.fs, T_init * units.kB, 0.01)
dyn = Bussi(m_g_na_relaxed_omat, 1 * units.fs, T_init, 0.01)
#n_steps = 5000 + 20000 # 5ps pre-equilibration + 20ps production
n_steps = 1000

dyn.attach(printenergy, interval = 100)
#dyn.attach(lambda: remove_COM_momentum(m_g_relaxed), interval=10)

traj = Trajectory('mg_na_1000.traj', 'w', m_g_na_relaxed_omat)
dyn.attach(traj.write, interval = 10)
printenergy()
dyn.run(n_steps)

Energy per atom: Epot = -9.204eV  Ekin = 0.038eV (T=293K)  Etot = -9.166eV Time Elapsed: 0m 1.0s
Energy per atom: Epot = -9.204eV  Ekin = 0.038eV (T=293K)  Etot = -9.166eV Time Elapsed: 0m 2.0s
Energy per atom: Epot = -9.169eV  Ekin = 0.044eV (T=337K)  Etot = -9.125eV Time Elapsed: 2m 58.0s
Energy per atom: Epot = -9.170eV  Ekin = 0.038eV (T=297K)  Etot = -9.132eV Time Elapsed: 5m 59.0s
Energy per atom: Epot = -9.164eV  Ekin = 0.038eV (T=294K)  Etot = -9.126eV Time Elapsed: 8m 55.0s
Energy per atom: Epot = -9.169eV  Ekin = 0.045eV (T=346K)  Etot = -9.124eV Time Elapsed: 11m 50.0s
Energy per atom: Epot = -9.160eV  Ekin = 0.035eV (T=274K)  Etot = -9.125eV Time Elapsed: 15m 0.0s
Energy per atom: Epot = -9.165eV  Ekin = 0.043eV (T=332K)  Etot = -9.122eV Time Elapsed: 18m 4.0s
Energy per atom: Epot = -9.158eV  Ekin = 0.037eV (T=283K)  Etot = -9.121eV Time Elapsed: 20m 52.0s
Energy per atom: Epot = -9.166eV  Ekin = 0.040eV (T=308K)  Etot = -9.126eV Time Elapsed: 23m 50.0s
Energy per atom: Ep

True

In [48]:
view(m_go_na_relaxed_omat, viewer='x3d')

### More than 1 Na ion

In [76]:
def add_sodium_many(heterostructure, n_Na = 8):
    # previously we added one Na ion to get 1/16 monolyaer (ML) coverage (16 O atoms, 1 Na)
    # now we want 1/2 ML coverage -> add 8 Na ions
    
    heterostructure = heterostructure.copy()
    
    graphene_C_atoms = [atom.position[2] for atom in heterostructure if atom.tag == 10]
    mxene_O_atoms = [atom.position[2] for atom in heterostructure if atom.tag == 3]
    
    z_mxene = np.mean(mxene_O_atoms)
    z_graphene = np.mean(graphene_C_atoms)

    # place Na ion halfway between MXene and graphene
    na_z = z_mxene + 0.5 * (z_graphene - z_mxene)

    # spread the 8 Na around evenly within the heterostructure
    positions = heterostructure.get_positions()
    xmin, ymin = np.min(positions[:, :2], axis=0)
    xmax, ymax = np.max(positions[:, :2], axis=0)

    # Generate n_Na evenly spaced positions in 2D
    na_positions = []
    for i in range(n_Na):
        x = xmin + (xmax - xmin) * (i / n_Na)  # Evenly spaced along x
        y = ymin + (ymax - ymin) * ((i % 2) / 2)  # Alternating in y direction
        na_positions.append([x, y, na_z])

    # Create Na atoms and add them to the heterostructure
    na_atoms = Atoms("Na" * n_Na, positions=na_positions)
    heterostructure += na_atoms  # Append Na to the structure

    return heterostructure

In [ ]:

def add_sodium_many(heterostructure, n_Na=8):
    """
    Adds `n_Na` sodium ions to the heterostructure in a hexagonal 3-2-3 pattern:
    - 3 Na in the top row
    - 2 Na in the middle row, shifted into gaps
    - 3 Na in the bottom row
    """

    heterostructure = heterostructure.copy()
    
    # Get z-coordinates of graphene (C atoms) and MXene (O atoms)
    graphene_C_atoms = [atom.position[2] for atom in heterostructure if atom.tag == 10]
    mxene_O_atoms = [atom.position[2] for atom in heterostructure if atom.tag == 3]
    
    z_mxene = np.mean(mxene_O_atoms)
    z_graphene = np.mean(graphene_C_atoms)

    # Place Na halfway between MXene and graphene
    na_z = z_mxene + 0.5 * (z_graphene - z_mxene)

    # Get unit cell boundaries
    positions = heterostructure.get_positions()
    xmin, ymin = np.min(positions[:, :2], axis=0)
    xmax, ymax = np.max(positions[:, :2], axis=0)

    # Define y-positions for the three rows
    y_positions = np.linspace(ymin+0.5, ymax-0.5, 3)  # Three distinct rows

    # Define x-positions for each row (evenly spaced)
    x_positions_top = np.linspace(xmin, xmax, 3, endpoint=False)  # 3 in top row
    x_positions_bottom = np.linspace(xmin+5, xmax+5, 3, endpoint=False)  # 3 in bottom row
    
    # Middle row: Offset x-coordinates to be in between the top and bottom row Na positions
    x_spacing = (x_positions_top[1] - x_positions_top[0]) / 2  # Half the x-spacing of top row
    x_positions_middle = x_positions_top[:-1] + x_spacing  # Shifted to the gaps

    # Construct Na positions
    na_positions = []
    
    # Top row (3 Na)
    for x in x_positions_top:
        na_positions.append([x, y_positions[2], na_z])  # Top row (highest y)

    # Middle row (2 Na) - staggered
    for x in x_positions_middle:
        na_positions.append([x, y_positions[1], na_z])  # Middle row (offset x)

    # Bottom row (3 Na)
    for x in x_positions_bottom:
        na_positions.append([x, y_positions[0], na_z])  # Bottom row (lowest y)

    # Create Na atoms and add them to the heterostructure
    na_atoms = Atoms("Na" * n_Na, positions=na_positions)
    heterostructure += na_atoms  # Append Na to the structure

    return heterostructure


In [104]:
m_g_na_many = add_sodium_many(m_g_relaxed_omat)
m_go_na_many = add_sodium_many(m_go_relaxed_omat)
m_goh_na_many = add_sodium_many(m_goh_relaxed_omat)

In [107]:
view(m_goh_na_many, viewer='x3d')

In [108]:
m_g_na_many_relaxed = m_g_na_many.copy()
m_g_na_many_relaxed.calc = macemp_omat
optimiser = BFGS(m_g_na_many_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 18:19:53    -1204.506500       12.000477
BFGS:    1 18:19:56    -1212.574759        5.877330
BFGS:    2 18:19:59    -1217.615200        3.379134
BFGS:    3 18:20:01    -1220.100197        2.256746
BFGS:    4 18:20:04    -1221.595064        2.066966
BFGS:    5 18:20:06    -1222.671579        2.251433
BFGS:    6 18:20:08    -1223.432687        2.146314
BFGS:    7 18:20:10    -1224.003855        1.505336
BFGS:    8 18:20:12    -1224.450964        1.262272
BFGS:    9 18:20:14    -1224.886390        1.551477
BFGS:   10 18:20:17    -1225.159952        1.328995
BFGS:   11 18:20:19    -1225.370103        0.881575
BFGS:   12 18:20:21    -1225.568704        0.577258
BFGS:   13 18:20:24    -1225.706552        0.484155
BFGS:   14 18:20:27    -1225.798746        0.476569
BFGS:   15 18:20:30    -1225.890385        0.547372
BFGS:   16 18:20:32    -1225.983125        0.493088
BFGS:   17 18:20:34    -1226.068335        0.465958
BFGS:   18 18:

True

In [109]:
io.write("structures/m_g_na_many_relaxed_omat.xyz", m_g_na_many_relaxed)

In [112]:
m_g_na_many_relaxed = io.read("structures/m_g_na_many_relaxed_omat.xyz")
view(m_g_na_many_relaxed, viewer='x3d')

In [110]:
m_go_na_many_relaxed = m_go_na_many.copy()
m_go_na_many_relaxed.calc = macemp_omat
optimiser = BFGS(m_go_na_many_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 18:27:05    -1187.849655       37.669235
BFGS:    1 18:27:07    -1203.874330       17.348671
BFGS:    2 18:27:09    -1215.142470        8.586554
BFGS:    3 18:27:11    -1220.611735        4.812927
BFGS:    4 18:27:13    -1223.752498        3.144833
BFGS:    5 18:27:15    -1225.777045        2.270227
BFGS:    6 18:27:17    -1227.148489        1.623214
BFGS:    7 18:27:19    -1228.042511        1.436449
BFGS:    8 18:27:21    -1228.631919        2.285304
BFGS:    9 18:27:24    -1229.183072        1.890656
BFGS:   10 18:27:26    -1229.650471        1.037811
BFGS:   11 18:27:28    -1229.936369        0.818146
BFGS:   12 18:27:31    -1230.241605        0.857988
BFGS:   13 18:27:33    -1230.394804        0.866777
BFGS:   14 18:27:35    -1230.588307        0.900915
BFGS:   15 18:27:37    -1230.785962        0.772757
BFGS:   16 18:27:39    -1230.957158        0.570811
BFGS:   17 18:27:41    -1231.094567        0.585712
BFGS:   18 18:

True

In [111]:
io.write("structures/m_go_na_many_relaxed_omat.xyz", m_go_na_many_relaxed)

In [113]:
m_go_na_many_relaxed = io.read("structures/m_go_na_many_relaxed_omat.xyz")
view(m_go_na_many_relaxed, viewer='x3d')

In [115]:
m_goh_na_many_relaxed = m_goh_na_many.copy()
m_goh_na_many_relaxed.calc = macemp_omat
optimiser = BFGS(m_goh_na_many_relaxed)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 18:41:13    -1214.212659       11.788816
BFGS:    1 18:41:16    -1222.193266        5.791653
BFGS:    2 18:41:18    -1227.222989        3.225179
BFGS:    3 18:41:20    -1229.768073        2.199473
BFGS:    4 18:41:23    -1231.179730        1.909006
BFGS:    5 18:41:26    -1232.240589        1.657049
BFGS:    6 18:41:28    -1232.986063        2.106513
BFGS:    7 18:41:30    -1233.537450        1.618115
BFGS:    8 18:41:33    -1234.005454        1.121109
BFGS:    9 18:41:35    -1234.366312        0.969482
BFGS:   10 18:41:37    -1234.632714        1.099243
BFGS:   11 18:41:39    -1234.791392        0.880427
BFGS:   12 18:41:41    -1234.941211        0.517616
BFGS:   13 18:41:43    -1235.068180        0.416124
BFGS:   14 18:41:45    -1235.153499        0.392413
BFGS:   15 18:41:48    -1235.225356        0.406160
BFGS:   16 18:41:50    -1235.299308        0.423501
BFGS:   17 18:41:52    -1235.369314        0.389529
BFGS:   18 18:

True

In [116]:
io.write("structures/m_goh_na_many_relaxed_omat.xyz", m_goh_na_many_relaxed)

In [117]:
m_goh_na_many_relaxed = io.read("structures/m_goh_na_many_relaxed_omat.xyz")
view(m_goh_na_many_relaxed, viewer='x3d')